In [2]:
from pydantic import BaseModel

#繼承BaseModel
#建立Person的class
class Person(BaseModel):
    first_name:str
    last_name:str
    age:int

In [3]:
#實作p1得Person
p1 = Person(first_name='John', last_name='Smith', age=42)
print(p1.first_name)
print(p1.last_name)
print(p1.age)
print(p1)

John
Smith
42
first_name='John' last_name='Smith' age=42


In [4]:
#自動字串 '42'轉換為 整數 42
p2 = Person(first_name='John', last_name='Smith', age='42')
p2

Person(first_name='John', last_name='Smith', age=42)

In [4]:
#使用錯誤資料，自動驗證，使用try  except判斷
try:
    p3 = Person(first_name='John', last_name='Smith', age='42a')
    print(p3)
except Exception as e:
    print(e)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='42a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


In [6]:
#使用dict轉換為BaseModel，建立p4的Person
#若為字串'42'時，也會自動轉換為整數 42 
data:dict = {
    "first_name": "John",
    "last_name": "Smith",
    "age":42,
}

p4 = Person.model_validate(data)
p4


Person(first_name='John', last_name='Smith', age=42)

In [7]:
# 3個'代表內容為字串
#將文字轉為BaseModel，建立p4的Person
data_json:str = '''
{
    "first_name": "John",
    "last_name": "Smith",
    "age":42
}
'''

#使用實體方法Person.model_validate_json將文字的json轉換為p5的P5的Person
p5 = Person.model_validate_json(data_json)
p5

Person(first_name='John', last_name='Smith', age=42)

In [8]:
#有預設值
class Person(BaseModel):
    first_name:str #require
    last_name:str | None = None #optional
    age:int = 18 #optional


#建立data_json時，沒有設定的內容時，以預設值代入
data_json:str = '''
{
    "first_name": "John"   
}
'''

p6 = Person.model_validate_json(data_json)
p6


Person(first_name='John', last_name=None, age=18)

In [9]:
#定義field內有list資料，一樣可以解析
class Person(BaseModel):
    first_name:str 
    last_name:str | None = None 
    age:int = 18
    lucky_numbers:list[int]

data_json:str = '''
{
    "first_name": "John",
    "last_name": "Smith",
    "age":42,
    "lucky_numbers":[1,3,5,7,9]
}
'''
p7 = Person.model_validate_json(data_json)
p7

Person(first_name='John', last_name='Smith', age=42, lucky_numbers=[1, 3, 5, 7, 9])

In [12]:
from pydantic import BaseModel,Field
#資料內容有id(會有衝突)、空白 、全部大寫
data:str = '''
{
    "id":100,
    "First Name":"John",
    "LASTNAME":"Smith",
    "age in years": 42
}
'''

#轉換資料
#使用Field重新定義欄位名稱及別名id去抓資料內容的key
#預設值使用default=18
class Person(BaseModel):
    id_:int = Field(alias="id")
    first_name:str = Field(alias='First Name')
    last_name:str | None = Field(alias='LASTNAME',default=None)
    age:int = Field(alias='age in years',default=18)

p8 = Person.model_validate_json(data)
p8

Person(id_=100, first_name='John', last_name='Smith', age=42)

In [13]:
#資料內容，巢狀結構
data = '''
{
    "firstName": "Arthur",
    "lastName": "Clarke",
    "born":{
        "place":{
            "country":"Lunar Colony",
            "city": "Central City"
        },
        "date":"2001-01-01"
    }
}
'''

#重新解析欄位，由最底層開始設定
class Place(BaseModel):
    country:str
    city:str

class Born(BaseModel):
    place:Place
    date:str

class Person(BaseModel):
    first_name:str = Field(alias='firstName')
    last_name:str = Field(alias='lastName')
    born:Born

p9 = Person.model_validate_json(data)
print(p9)
p9.born.place.city
p9.born.place.country

first_name='Arthur' last_name='Clarke' born=Born(place=Place(country='Lunar Colony', city='Central City'), date='2001-01-01')


'Lunar Colony'